In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import fastai
fastai.__version__

'1.0.37'

In [3]:
from fastai import *
from fastai_audio import *

In [4]:
import librosa
from utils import read_file
from tqdm import tqdm_notebook as tqdm

In [5]:
DATA = Path('data')
NSYNTH_AUDIO = DATA/'nsynth_audio' # contains train and valid folders

In [6]:
def save_frames_df(df, name):
    frames_np = np.stack(df['frame'].values)
    np.save(name + '.npy', frames_np)
    df[['note_str', 'pitch']].to_csv(name + '.csv', index=False)
    
def load_frames_df(name):
    df = pd.read_csv(name + '.csv')
    frames_np = np.load(name + '.npy')
    list_frames = np.array_split(frames_np, frames_np.shape[0])
    df['frame'] = pd.Series(map(np.squeeze, list_frames))
    return df

In [7]:
trn_frames = load_frames_df('trn_frames_1024')
val_frames = load_frames_df('val_frames_1024')
tst_frames = load_frames_df('tst_frames_1024')

In [67]:
len(trn_frames), len(val_frames), len(tst_frames)

(440752, 32098, 10040)

In [9]:
# df = trn_frames.iloc[:10].copy()
df = trn_frames.copy()
# df = df.sample(frac=1).reset_index(drop=True)
df = df.iloc[:10000]
df.head(3)

,note_str,pitch,frame
0,guitar_acoustic_001-082-050,82,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,guitar_acoustic_001-082-050,82,"[0.20001832, 0.22281563, 0.18192084, 0.0400402..."
2,guitar_acoustic_001-082-050,82,"[-0.036896877, 0.047364727, 0.16974396, 0.2450..."


In [14]:
def normalize(signal, amp=1.0):
    return signal / np.abs(signal).max() * amp

In [60]:
def poly_frames(df, max_notes=4):
    df = df.copy()
    new_rows = []
    while True:
        try:
            n_notes = np.random.randint(1, max_notes+1)
            rows = []
            pitches = []
            for i in range(n_notes):
                row = df[~df['pitch'].isin(pitches)].sample(1)
                df.drop(row.index, inplace=True)
                rows.append(row)
                pitches.append(row.iloc[0]['pitch'])
            rows_df = pd.concat(rows)
            pitches = rows_df['pitch'].values
            frames = np.stack(rows_df['frame'].values)
            frame_sum = frames.sum(axis=0)
            max_amp = np.max(np.abs(frames))
            frame = normalize(frame_sum, max_amp)

            new_row = (pitches, len(pitches), frame)
            new_rows.append(new_row)
        except Exception as e:
            break
              
    df_poly = pd.DataFrame(new_rows, columns=['pitches', 'n_pitches', 'frame'])
    return df_poly

In [104]:
def poly_frames_chunks(df, max_notes=4, chunk_size=10000):
    df_all = df.copy()
    new_rows = []
    
    n_chunks = 0
    while len(df_all) > 0:
        df = df_all.iloc[:chunk_size]
        df_all = df_all.iloc[chunk_size:]
        while True:
            n_notes = np.random.randint(1, max_notes+1)
            rows = []
            pitches = []
            for i in range(n_notes):
                valid_rows = df[~df['pitch'].isin(pitches)]
                # all the remaining rows are the same pitch.. 
                if len(valid_rows) == 0:
                    break
                row = valid_rows.sample(1)
                df.drop(row.index, inplace=True)
                rows.append(row)
                pitches.append(row.iloc[0]['pitch'])
            # no rows were built
            if len(rows) == 0:
                break
            rows_df = pd.concat(rows)
            pitches = rows_df['pitch'].values
            frames = np.stack(rows_df['frame'].values)
            frame_sum = frames.sum(axis=0)
            max_amp = np.max(np.abs(frames))
            frame = normalize(frame_sum, max_amp)

            new_row = (pitches, len(pitches), frame)
            new_rows.append(new_row)

        n_chunks += 1
        print("Completed {} chunks".format(n_chunks))
                              
    df_poly = pd.DataFrame(new_rows, columns=['pitches', 'n_pitches', 'frame'])
    return df_poly

In [109]:
trn_frames_poly_c = poly_frames_chunks(trn_frames)

Completed 1 chunks
Completed 2 chunks
Completed 3 chunks
Completed 4 chunks
Completed 5 chunks
Completed 6 chunks
Completed 7 chunks
Completed 8 chunks
Completed 9 chunks
Completed 10 chunks
Completed 11 chunks
Completed 12 chunks
Completed 13 chunks
Completed 14 chunks
Completed 15 chunks
Completed 16 chunks
Completed 17 chunks
Completed 18 chunks
Completed 19 chunks
Completed 20 chunks
Completed 21 chunks
Completed 22 chunks
Completed 23 chunks
Completed 24 chunks
Completed 25 chunks
Completed 26 chunks
Completed 27 chunks
Completed 28 chunks
Completed 29 chunks
Completed 30 chunks
Completed 31 chunks
Completed 32 chunks
Completed 33 chunks
Completed 34 chunks
Completed 35 chunks
Completed 36 chunks
Completed 37 chunks
Completed 38 chunks
Completed 39 chunks
Completed 40 chunks
Completed 41 chunks
Completed 42 chunks
Completed 43 chunks
Completed 44 chunks
Completed 45 chunks


In [110]:
save_poly_frames_df(trn_frames_poly_c, 'trn_frames_poly_1024_4')

In [111]:
len(trn_frames_poly_c)

176228

In [112]:
len(trn_frames)

440752

In [113]:
trn_frames_poly_c['n_pitches'].sum()

440752

In [ ]:
trn_frames_poly_c

In [103]:
45 total chunks.. 

440.752

In [61]:
%%time
tst_frames_poly = poly_frames(tst_frames)

CPU times: user 29.5 s, sys: 604 ms, total: 30.1 s
Wall time: 30.1 s


In [62]:
save_poly_frames_df(tst_frames_poly, 'tst_frames_poly_1024_4')

In [70]:
%%time
val_frames_poly = poly_frames(val_frames)

CPU times: user 2min 27s, sys: 13 s, total: 2min 40s
Wall time: 2min 40s


In [71]:
save_poly_frames_df(val_frames_poly, 'val_frames_poly_1024_4')

In [72]:
%%time
trn_frames_poly = poly_frames(trn_frames)

KeyboardInterrupt: 

In [73]:
save_poly_frames_df(trn_frames_poly, 'trn_frames_poly_1024_4')

NameError: name 'trn_frames_poly' is not defined

In [63]:
df1 = load_poly_frames_df('tst_frames_poly_1024_4')

In [68]:
df1.head()

,pitches,n_pitches,frame
0,[43],1,"[0.020447401, 0.021607105, 0.022370068, 0.0228..."
1,"[69, 60]",2,"[0.42534202, 0.42985627, 0.4050668, 0.3499934,..."
2,"[45, 53, 56]",3,"[0.15157439, 0.18534634, 0.21727508, 0.2458859..."
3,"[82, 42, 65, 51]",4,"[-0.29671514, -0.32114583, -0.32809636, -0.315..."
4,"[79, 82]",2,"[0.045332216, 0.116490625, 0.17688969, 0.21778..."


In [51]:
def save_poly_frames_df(df, name):
    frames_np = np.stack(df['frame'].values)
    np.save(name + '.npy', frames_np)
    df[['pitches', 'n_pitches']].to_csv(name + '.csv', index=False)
    
def load_poly_frames_df(name):
    df = pd.read_csv(name + '.csv')
    # convert string column back to list of ints
    df['pitches'] = df['pitches'].apply(lambda s: list(map(int, s[1:-1].split(' '))))
    frames_np = np.load(name + '.npy')
    list_frames = np.array_split(frames_np, frames_np.shape[0])
    df['frame'] = pd.Series(map(np.squeeze, list_frames))
    return df

In [ ]:
from scipy.signal import get_window

In [ ]:
def get_pow_db(x):
    w = get_window('hann', x.shape[0])
    wx = x*w
    X = np.fft.rfft(wx)
    Xpow = abs(X) #** 2
    return librosa.amplitude_to_db(Xpow, ref=1.0, top_db=50.0)

In [ ]:
fig, ax = plt.subplots(4,2, figsize=(16, 6))
for i in range(3):
    x = frames[i]
    ax[i][0].plot(x)
    ax[i][1].plot(get_pow_db(x))
ax[3][0].plot(frame_sum)
ax[3][1].plot(get_pow_db(frame_sum))

In [ ]:
pitches

In [ ]:
frame_sum.min(), frame_sum.max()

In [ ]:
np.max(abs(frames), axis=1)

In [ ]:
plt.figure(figsize=(6, 1))
plt.plot(normalize(frame_sum, 0.57));

In [ ]:
dfb

In [ ]:
num = 3

rows = []
for i in range(3):
    row = df.sample(1)
    df.drop(row.index, inplace=True)
    rows.append(row)